# Proyecto Final Algebra Lineal 2017
---

El proyecto del curso consiste en la implementación del [Algortimo de Canny](https://es.wikipedia.org/wiki/Algoritmo_de_Canny) para la detección de bordes en imágenes y videos. Para ello, se le entrega al estudiante toda la implementación del algoritmo en [OpenCV](http://opencv.org/) y su objetivo es **implementar las funciones de OpenCV con matrices**. 

Las funciones a modificar serán las siguientes:
```python
cv2.GaussianBlur()
cv2.Sobel()
```

No es necesario que modifique más del código que se le solicita, pero puede hacer las modificaciones pertinentes si ese fuera el caso. Dentro del código se le indica en qué parte debe de ser modificado, por ejemplo:

```python
'''Modificar esta función'''
    sobel = cv2.Sobel(img, cv2.CV_64F, x, y, ksize=sobel_kernel)
``` 

Se recomienda buscar maneras de optimizar los cálculos de sus filtros, pues más adelante existe una aplicación en tiempo real donde es crucial que sus cálculos sean lo más rápidos posibles. Las dimensiones de los *kernels* no pueden ser modificadas.

## 0. Conceptos
Para poder modificar las funciones solicitadas es necesario que comprenda algunos conceptos, por lo cual es necesario que defina:

1. ¿En que consiste la convolución y cómo puede ser calculada? (Puede dar una definición matemática y/o ejemplos).

2. ¿Qué es el *kernel* de un filtro y para que se utiliza?

3. ¿En qué consiste un filtro Gaussiano?

4. ¿En qué consiste el Filtro Sobel? ¿para qué sirve?

3. Describa el algoritmo de detección de bordes Canny.

In [1]:
import numpy as np
import matplotlib.pylab as plt
import cv2
import math
import time

from PIL import Image
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

## Methods block

In [2]:
def gaussian_kernel():
    '''My function'''
    kernel = np.matrix([[1,4,7,4,1],[4,20,33,20,4],[7,33,55,7,33],[4,20,33,20,4],[1,4,7,4,1]])*0.003663004
    #kernel = np.matrix([[1,4,7,4,1],[4,16,26,16,4],[7,26,41,26,7],[4,16,26,16,4],[1,4,7,4,1]])
    return kernel

def sobelx_kernel():
    '''My function'''
    #kernel = np.matrix([[-3,0,3],[-10,0,10],[-3,0,3]])
    #kernel = np.matrix([[-5,0,5],[-12, 0, 12],[-5,0,5]])
    kernel = np.matrix([[1, 0, -1],[2, 0, -2], [1, 0, -1]])
    #kernel = np.matrix([[-1, 0,+1],[-2,0,2],[-1,0,1]])
    return kernel
def sobely_kernel():
    '''My function'''
    #kernel = np.matrix([[-3,-10,-3],[0,0,0],[3,10,3]])
    #kernel = np.matrix([[-5, -12,-5],[0,0,0],[5,12,5]])
    kernel = np.matrix([[1, 2, 1], [0, 0, 0],[-1, -2, -1]])
    #kernel = np.matrix([[1, 2,1],[0,0,0],[-1,-2,-2]])
    return kernel


def gaussian_blur(img, kernel=5):
    return convolution(img, gaussian_kernel()) #return cv2.GaussianBlur(img,(kernel,kernel),0)
    
    
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    '''Modificar esta función'''
    #sobel = cv2.Sobel(img, cv2.CV_64F, x, y, ksize=sobel_kernel) 
    x_kernel = sobelx_kernel()
    y_kernel = sobely_kernel()

    
    if orient == 'x':   
        sobel = conv1_sobel(img.shape[1] + 2, img.shape[0] + 2, img, np.array(x_kernel[0:3,0:1]), np.array(x_kernel[0:1,0:3])[0])
        #sobel = convolution(img, sobelx_kernel())
    elif orient == 'y':
        sobel = conv1_sobel(img.shape[1] + 2, img.shape[0] + 2, img, np.array(y_kernel[0:3,0:1]), np.array(y_kernel[0:1,0:3])[0])
        #sobel = convolution(img, sobely_kernel())
            
    
    # Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)

    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # Create a mask of 1's where the scaled gradient magnitude 
    # is > thresh_min and < thresh_max
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sbinary

def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    # Take the gradient in x and y separately
    
    #sobel_results = double_convolution(image, sobelx_kernel(), sobely_kernel())
    x_kernel = sobelx_kernel()
    y_kernel = sobely_kernel()
    
    '''Modificar esta función'''
   #sobelx = convolution(image, sobelx_kernel()) #sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
   #sobelx = sobel_results[0]
    sobelx =conv1_sobel(image.shape[1] + 2, image.shape[0] + 2, image, np.array(x_kernel[0:3,0:1]), np.array(x_kernel[0:1,0:3])[0])
    '''Modificar esta función'''
    #sobely = convolution(image, sobely_kernel()) #sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    #sobely = sobel_results[1]
    sobely = conv1_sobel(image.shape[1] + 2, image.shape[0] + 2, image, np.array(y_kernel[0:3,0:1]), np.array(y_kernel[0:1,0:3])[0])
    # Calculate the magnitude 
    abs_sobel = np.sqrt(sobelx**2 +sobely**2)

    # Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # Create a binary mask where mag thresholds are met
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    return sbinary

def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    # Take the gradient in x and y separately 
    
    #sobel_results = double_convolution(image, sobelx_kernel(), sobely_kernel())
    x_kernel = sobelx_kernel()
    y_kernel = sobely_kernel()
    
    '''Modificar esta función'''
    #sobelx = convolution(image, sobelx_kernel()) #sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    #sobelx = sobel_results[0]
    sobelx =conv1_sobel(image.shape[1] + 2, image.shape[0] + 2, image, np.array(x_kernel[0:3,0:1]), np.array(x_kernel[0:1,0:3])[0])
    '''Modificar esta función'''
    #sobely = convolution(image, sobely_kernel())#sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    #sobely = sobel_results[1]
    sobely = conv1_sobel(image.shape[1] + 2, image.shape[0] + 2, image, np.array(y_kernel[0:3,0:1]), np.array(y_kernel[0:1,0:3])[0])
    
    # Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)   
    abs_sobely = np.absolute(sobely)
    
    # Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    grad_dir = np.arctan2(abs_sobely, abs_sobelx)
    
    # Create a binary mask where direction thresholds are met
    sbinary = np.zeros_like(grad_dir)
    sbinary[(grad_dir >= thresh[0]) & (grad_dir <= thresh[1])] = 1
    return sbinary

## Convolution methods

In [3]:
#methods mainly for sobel filter

def conv1_sobel(w, h, img, convc, convr):
    #this method uses a vertical and horizontal R1 kernel
    image = np.zeros(shape=(w, h), dtype=int)
    for m in range(0, w - 3):
        for n in range(0, h - 3):
            image[m + 1, n + 1] = img[n,m]
        
    image2 = np.zeros(shape=(w, h), dtype=int)
    image2 = np.array(image2)
    for x in range(1, w - 2):
        for y in range(1, h - 2):
            image2[x, y] = (image[x - 1, y] * convc[0, 0] + image[x, y] * convc[1,0] + image[x + 1, y] * convc[2, 0])
    image3 = np.zeros(shape=(w-2, h-2), dtype=int)
    image3 = np.array(image3)
    for x in range(1, w - 2):
        for y in range(1, h - 2):
            image3[x-1, y-1] = (image2[x, y-1] * convr[0] + image2[x, y] * convr[1] + image2[x, y+1] * convr[2])
            if (image3[x, y] < 0):
                image3[x, y] =-image3[x,y]
    return np.rot90(image3,3)

def conv2_sobel(w, h, img, conv):
    #this method uses a conventional kernel, its operations are not done with methods of some class
   
    image = np.zeros(shape=(w, h), dtype=int)
    for m in range(0, w - 3):
        for n in range(0, h - 3):
            image[m + 1, n + 1] = img[n,m]
        
    image2 = np.zeros(shape=(w-2, h-2), dtype=int)
    image2 = np.array(image2)
    for x in range(1, w - 2):
        for y in range(1, h - 2):
            image2[x-1, y-1] = (image[x - 1, y - 1] * conv[0, 0] + image[x - 1, y] * conv[0, 1] + image[x - 1, y + 1] * conv[0, 2] +
                                image[x, y - 1] * conv[1, 0] + image[x, y] * conv[1, 1] + image[x, y + 1] * conv[1, 2] +
                                image[x + 1, y - 1] * conv[2, 0] + image[x + 1, y] * conv[2, 1] + image[x + 1, y + 1] * conv[2, 2])
            if (image2[x, y] < 0):
                image2[x, y] =-image2[x,y]
    return np.rot90(image2,3)

def conv3_sobel(img, kernelx, kernely):
    #this method returns sobely and sobelx at the same time
    #it assume the kernel is square and dim(kernelx) = dim(kernely)
    difference = int(math.floor(kernelx.shape[0]/2))
    #x elements
    kernelx_sum = np.sum(np.abs(kernelx))
    img_x = np.zeros((img.shape[0]-difference, img.shape[1]-difference))
    #y elements
    kernely_sum = np.sum(np.abs(kernely))
    img_y = np.zeros((img.shape[0]-difference, img.shape[1]-difference))
    
    for row in range(difference, img.shape[0]-difference):
        for column in range(difference, img.shape[1]-difference):
            #create the current pixel matrix
            current_matrix = np.matrix(img[row-difference:row+difference+1, column-difference: column+difference+1])
            #current matrix is multiplicated with the kernel, element to element
            currentx = np.multiply(current_matrix, kernelx)
            currenty = np.multiply(current_matrix,kernely)
            #summing current matrix elements
            sum_x = np.sum(currentx)
            sum_y = np.sum(currenty)
            #dividing the current sum and kernel sum and sustituded and replacing the pixel value
            img_x[row-difference, column-difference] = sum_x/kernelx_sum
            img_y[row-difference, column-difference] = sum_y/kernely_sum
    return np.array([img_x,img_y])

def convolution(img, kernel):
    #It is a conventional convolution, this method was used for the gaussian kernel, 
    #but also was tested for the sobel filter
    difference = int(math.floor(kernel.shape[0]/2))
    new_img = np.zeros((img.shape[0]-difference, img.shape[1]-difference))
    kernel_sum = np.sum(np.abs(kernel))
    for row in range(difference, img.shape[0]-difference):
        for column in range(difference, img.shape[1]-difference):
            #create the current pixel matrix
            current_matrix = np.matrix(img[row-difference:row+difference+1, column-difference: column+difference+1])
            #current matrix is multiplicated with the kernel, element to element
            current_matrix = np.multiply(current_matrix, kernel)
            #summing current matrix elements
            current_sum = np.sum(current_matrix)
            #dividing the current sum and kernel sum and sustituded and replacing the pixel value
            new_img[row-difference, column-difference] = current_sum/kernel_sum
    return new_img

## Time tester (convolution methods)

In [ ]:
#tester
i = Image.open('Gato.jpg')
im = cv2.cvtColor(np.array(i), cv2.COLOR_RGB2GRAY)
width,height = i.size
      
x_kernel = sobelx_kernel()
y_kernel = sobely_kernel()

#....method 1
time_start = time.time()
G1dx=conv1_sobel(width + 2, height + 2, im, np.array(x_kernel[0:3,0:1]), np.array(x_kernel[0:1,0:3])[0])
G1dy=conv1_sobel(width + 2, height + 2, im, np.array(y_kernel[0:3,0:1]), np.array(y_kernel[0:1,0:3])[0])
time_end = time.time()
print('Conv1S function: %0.3f ms' %((time_end-time_start)*1000.0))
#... method 2
time_start = time.time()
G2dx=conv2_sobel(width + 2, height + 2, im, x_kernel)
G2dy=conv2_sobel(width + 2, height + 2, im, y_kernel)
time_end = time.time()
print('Conv2S function: %0.3f ms' %((time_end-time_start)*1000.0))
#... method 3
time_start = time.time()
sobelxy = conv3_sobel(im, x_kernel, y_kernel)
G2dx=sobelxy[0]
G2dy=sobelxy[1]
time_end = time.time()
print('Conv3S function: %0.3f ms' %((time_end-time_start)*1000.0))
#... method 4
time_start = time.time()

G2dx = convolution(im, x_kernel)
G2dy = convolution(im, y_kernel)
time_end = time.time()
print('Conv function: %0.3f ms' %((time_end-time_start)*1000.0))

## 1. Prueba en Imagen

In [ ]:
# Test filter on single image
im = plt.imread("BTD.jpg")
gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)

# Gaussian Blur Filtering
gray = gaussian_blur(gray, kernel = 5)

# Choose a Sobel kernel size
ksize = 3 # Choose a larger odd number to smooth gradient measurements

# Apply each of the thresholding functions
gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=ksize, thresh=(50, 100))
grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=ksize, thresh=(50, 100))
mag_binary = mag_thresh(gray, sobel_kernel=ksize, mag_thresh=(20, 100))
dir_binary = dir_threshold(gray, sobel_kernel=ksize, thresh=(0, np.pi/2))

# Create combined image with sobel filters applied
combined = np.zeros_like(dir_binary)
combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1

rgb_combined = np.asarray(np.dstack((combined, combined, combined)), dtype=np.float64)

In [ ]:
# Plot the result
f, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(10, 15))
f.tight_layout()
ax1.imshow(im)
ax1.set_title('Original Image', fontsize=20)
ax2.imshow(gray, cmap='gray')
ax2.set_title('Gray Image', fontsize=20)
ax3.imshow(rgb_combined, cmap='gray')
ax3.set_title('Combined Sobel Image', fontsize=20)
ax4.imshow(gradx, cmap='gray')
ax4.set_title('GradX Image', fontsize=20)
ax5.imshow(grady, cmap='gray')
ax5.set_title('GradY Image', fontsize=20)
ax6.imshow(((mag_binary == 1) & (dir_binary == 1)), cmap='gray')
ax6.set_title('Thresholded Grad. Dir.', fontsize=20)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.show() 

## 2. Prueba en Videos

In [4]:
def process_image(image):
    
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = gaussian_blur(gray, kernel = 5)
    # Choose a Sobel kernel size
    ksize = 3 # Choose a larger odd number to smooth gradient measurements

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=ksize, thresh=(50, 100))
    grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=ksize, thresh=(50, 100))
    mag_binary = mag_thresh(gray, sobel_kernel=ksize, mag_thresh=(20, 100))
    dir_binary = dir_threshold(gray, sobel_kernel=ksize, thresh=(0, np.pi/4))
    
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    
    rgb_combined = np.asarray(np.dstack((combined, combined, combined)), dtype=np.uint8)
    
    retval, threshold = cv2.threshold(rgb_combined, 0, 255, cv2.THRESH_BINARY)
    
    return threshold

In [5]:
time_start = time.time()
filter_output = 'Tambor_Filter.mp4'
clip1 = VideoFileClip("Tambor.mp4")
batman_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time batman_clip.write_videofile(filter_output, audio=False)

time_end = time.time()
print('Conv function: %0.3f ms' %((time_end-time_start)*1000.0))

[MoviePy] >>>> Building video Tambor_Filter.mp4
[MoviePy] Writing video Tambor_Filter.mp4
[MoviePy] Done.                                                            
[MoviePy] >>>> Video ready: Tambor_Filter.mp4 

Wall time: 2h 38min 17s
Conv function: 9535041.000 ms


In [6]:
HTML("""
<video width="720" height="480" controls>
  <source src="{0}">
</video>
""".format(filter_output))

In [ ]:
cap = cv2.VideoCapture(0)  # 0 for Python 3, 1 for Python 2. No idea why?!!

fourcc = cv2.cv.CV_FOURCC(*"MJPG")

#out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 20,(640,480))
out = cv2.VideoWriter("output.avi", fourcc, 20,(640,480))


while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = cv2.flip(frame,180)
        new_frame = process_image(frame)
        out.write(new_frame)

        cv2.imshow('frame',new_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()